In [3]:
%run COMPLEJO.ipynb
%run VECTOR_MATRIZ.ipynb

In [4]:
import unittest
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy import linalg as LA

In [1]:
class Systems:
    CLASSIC = 0
    PROBABILISTIC = 1
    QUANTUM = 2
    SLIT_QU=3
    SLIT_PROB=4
    
    INVALID_SYSTEM = "The system is not valid"
    INCORRECT_FORMAT = "The matriz doesn't match de system format"
    INVALID_VECTOR = "The vector is not valid"
    @staticmethod
    def verifySystem(matrix, systemType):
        ans = False
        if(systemType==Systems.CLASSIC):
            ans = Systems.verifyClassic(matrix)
        elif(systemType==Systems.PROBABILISTIC):
            ans = Systems.verifyProbabilistic(matrix)
        elif(systemType==Systems.QUANTUM):
            ans = Systems.verifyQuantum(matrix)
        else: 
            raise Exception(Systems.INVALID_SYSTEM)
        return ans
    
    @staticmethod
    def verifyClassic(matrix):
        isValid = True
        one = Complejo(1,0)
        zero = Complejo(0,0)
        columnSum = Complejo(0,0)
        j = 0
        while(j<matrix.columnas and isValid):
            i=0
            columnSum=Complejo(0,0)
            while(i<matrix.filas and isValid):
                isValid = isValid and (matrix[i,j]==one or matrix[i,j]==zero)
                columnSum+=matrix[i,j]
                i+=1
            isValid= isValid and columnSum==one
            j+=1
        return isValid
            
    @staticmethod
    def verificarColumnasEstocasticas(matrix):
        isValid = True
        one = Complejo(1,0)
        zero = Complejo(0,0)
        columnSum = Complejo(0,0)
        j = 0
        while(j<matrix.columnas and isValid):
            i=0
            while(i<matrix.filas and isValid):
                s = matrix[i,j].parteReal
                isValid = isValid and matrix[i,j].parteImaginaria==0 and s>=0 and s<=1
                columnSum+=matrix[i,j]
                i+=1
            isValid= isValid and columnSum==one
            columnSum=Complejo(0,0)
            j+=1
        
        return isValid
    
    @staticmethod
    def verificarFilasEstocasticas(matrix):
        isValid = True
        one = Complejo(1,0)
        zero = Complejo(0,0)
        columnSum = Complejo(0,0)
        i = 0
        while(i<matrix.filas and isValid):
            j=0
            while(j<matrix.columnas and isValid):
                s = matrix[i,j].parteReal
                isValid = isValid and matrix[i,j].parteImaginaria==0 and s>=0 and s<=1
                columnSum+=matrix[i,j]
                j+=1
            isValid= isValid and columnSum==one
            columnSum=Complejo(0,0)
            i+=1
        return isValid
    
    
    @staticmethod
    def verifyProbabilistic(matrix):
        return Systems.verificarFilasEstocasticas(matrix) and Systems.verificarColumnasEstocasticas(matrix)
    
    @staticmethod
    def verifyQuantum(matrix):
        return matrix.esUnitaria()
    
    
    @staticmethod
    def verifyVector(state0, systemType):
        ans= state0.columnas==1
        if(systemType==Systems.CLASSIC):
            ans = ans and Systems.verifyClassic(state0)
        elif(systemType==Systems.PROBABILISTIC):
            ans = ans and Systems.verificarColumnasEstocasticas(state0)
        elif(systemType==Systems.QUANTUM):
            prueba=Matriz(state0.filas,1)
            for i in range(state0.filas):
                prueba[i,0]=state0[i,0]*(state0[i,0].conjugado())
                
            
            ans = ans and Systems.verificarColumnasEstocasticas(prueba)
        else: 
            raise Exception(Systems.INVALID_VECTOR)
        return ans
    
    def __init__(self, systemType, matrix: Matriz, state0, clicks, statesA=None, statesB=None, assembled=False):
        self.state = [True, None, None]
        self.statesA=statesA
        self.statesB=statesB
        self.assembled=assembled
        self.type=None
        #[0]=isValid system, [1]=matriz^clicks, [2]=state0->final[t]
        if(systemType!=Systems.SLIT_QU and systemType!=Systems.SLIT_PROB and (not Systems.verifySystem(matrix, systemType) or matrix.filas==0 or matrix.filas!=matrix.columnas 
           or matrix.filas!=state0.filas)):
            self.state[0]=False
        else:
            
            self.state[1]=matrix**clicks
            self.state[2]=self.state[1]*state0
            self.type=systemType
   
    def success(self):
        return self.state[0]
    
    
    @staticmethod
    def obtainType(matrix):
        ans = -1
        if(Systems.verifyClassic(matrix)):
            ans = Systems.CLASSIC
        elif(Systems.verifyProbabilistic(matrix)):
            ans = Systems.PROBABILISTIC
        elif(Systems.verifyQuantum(matrix)):
            ans = Systems.QUANTUM
        return ans
    
    
    @staticmethod
    def assembleSystems(systemA, viA, systemB, viB, t):
        assembled = systemA.productoTensor(systemB)
        statei = viA.productoTensor(viB)
        return Systems(Systems.obtainType(systemA), assembled, statei, t, viA.filas, viB.filas, True)
    
    @staticmethod
    def fullSlits(matrix, slits, typeS):
        t=1/slits
        if(typeS==Systems.SLIT_QU):
            t=t**0.5
        for i in range(1, slits+1):
            matrix[i,0]=Complejo(t,0)
    
    @staticmethod
    def fullTargets(matrix, slits,targets, vector):
        begin=slits+1
        final=begin+2*targets+1
        
        for j in range(1, slits+1):
            for i in range(begin, final):
                
                matrix[i,j]=vector[i-begin,0]
                
            begin+=targets+1
            final=begin+2*targets+1
            
    @staticmethod
    def fullReturn(matrix, slits,targets):
        begin=slits+1
        for i in range(begin, matrix.filas):
            matrix[i,i]=Complejo(1,0)
    
    @staticmethod
    def doubleSlit(slits, targets, vector):
        prob=Systems.verifyVector(vector, Systems.PROBABILISTIC)
        quan=Systems.verifyVector(vector, Systems.QUANTUM)
        
        if(vector.filas!=2*targets+1 or vector.columnas!=1 or not (prob or quan)):
            raise Exception("Vector invalid")
        n=1+slits+(slits+targets*(slits+1))
        typeS=-1
        if(prob):
            typeS=Systems.SLIT_PROB
        else:
            typeS=Systems.SLIT_QU
        matrix=Matriz(n,n)
        Systems.fullSlits(matrix, slits, typeS)
        Systems.fullTargets(matrix, slits,targets, vector)
        Systems.fullReturn(matrix, slits, targets)
        state0=Matriz(n,1)
        state0[0,0]=Complejo(1,0)
        
        return Systems(typeS, matrix, state0, 2)
        
    def obtenerDatos(self):
        ans=[]
        if(self.type==Systems.QUANTUM or self.type==Systems.SLIT_QU):
            for i in range(self.state[2].filas):
                ans.append(self.state[2][i,0].modulo()**2)
        else:
            for i in range(self.state[2].filas):
                ans.append(self.state[2][i,0].parteReal)
        
        return ans
    def showGraphic(self):
        fig = plt.figure()
        
        ax=fig.add_subplot(111)
        datos=self.obtenerDatos()
        xx=range(len(datos))
        letter=[str(i) for i in range(len(datos))]
        ax.bar(xx, datos , color=(0,1,0), align="center")
        ax.set_xticks(xx)
        ax.set_xticklabels(letter)
        plt.show()
        
    @staticmethod
    def observableMedia(ket, observable):
        if(not observable.esHermitiana()):
            raise Exception("La matriz debe ser hermitiana")
        ket= ket*(1/ket.norma())
        return (observable*ket).productoInterno(ket)
    
    
    @staticmethod
    def observableVarianza(ket, observable):
        if(not observable.esHermitiana()):
            raise Exception("La matriz debe ser hermitiana")
        ket= ket*(1/ket.norma())
        media=(observable*ket).productoInterno(ket)
        ide=Matriz(observable.filas, observable.filas)
        for i in range(observable.filas):
            ide[i,i]=Complejo(1,0)
        gama=observable+ide*(media*-1)
        return (ket.adjunta()*(gama**2)*ket)[0,0]
    
    @staticmethod
    def valoresPropios(matrix):
        if(not matrix.esHermitiana()):
            raise Exception("La matriz debe ser hermitiana")
            #e_vals, e_vecs = LA.eig(A)
        A=[]
        for i in range(matrix.filas):
            t=[]
            for j in range(matrix.columnas):
                t.append(matrix[i,j].parteReal)
            A.append(t)
            
        return LA.eig(A)[0]
    
    @staticmethod
    def particulaProbabilidad(n, ket):
        fig = plt.figure()
        
        ax=fig.add_subplot(111)
        datos=[]
        norma=ket.norma()
        for i in range(ket.filas):
            datos.append(ket[i,0].modulo()**2/norma**2)
        xx=range(len(datos))
        letter=["x_"+str(i) for i in range(len(datos))]
        ax.bar(xx, datos , color=(0,1,0), align="center")
        ax.set_xticks(xx)
        ax.set_xticklabels(letter)
        plt.show()
        return datos
        

NameError: name 'Matriz' is not defined

In [17]:
#M = Matriz (3,3, [[0,1/6, 5/6], [1/3, 1/2, 1/6], [2/3, 1/3, 0]])
#a = Systems(2, M, Matriz(3,1,[[1/5],[7/10],[1/10]]), 4)
#print(a.state[2])

In [18]:
#animo=Matriz(2,2, [[1/3, 2/3], [2/3, 1/3]])
#posicion=Matriz(3,3, [[0,1/6, 5/6], [1/3, 1/2, 1/6], [2/3, 1/3,0]])
#a=Systems.assembleSystems(animo, Matriz(3,1,[[0.01],[ 0.9], [0.09]]), posicion, Matriz(2,1, [[0.05],[0.95]]), 3)
#print(a.state[2])